In [1]:
#%matplotlib inline
import numpy as np
from scipy.misc import imread
import pickle
import matplotlib.pyplot as plt
from scipy.misc import imread, imresize
import tensorflow as tf
from keras.preprocessing import image

from keras.backend.tensorflow_backend import set_session
from ssd import SSD300
from keras.applications.imagenet_utils import preprocess_input
from ssd_utils import BBoxUtility
import matplotlib.pyplot as plt
from SSD_tester import calc_detection_prec_rec, calc_detection_ap

config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list='1',
        allow_growth=True 
    )
)
sess = sess = tf.Session(config=config)



Using TensorFlow backend.


In [2]:
NYU_CLASSES = ['bathtub', 'bed', 'bookshelf', 'box', 'chair', 'counter', 'desk', 'door', 'dresser',
               'garbage_bin', 'lamp', 'monitor', 'night_stand', 'pillow', 'sink', 'sofa', 'table', 'tv', 'toilet']
NUM_CLASSES = len(NYU_CLASSES) + 1
input_shape = (300, 300, 3) #channel lastde
depth_input_shape = (300, 300, 1)

In [3]:
model = SSD300(input_shape, num_classes=NUM_CLASSES)
model.load_weights('/data/jun/checkpoints/SUNRGBD/estimation/RGB/v10/weights.48-2.47.hdf5')

(64,)


In [4]:
rgb_gt = pickle.load(open('../pkls/SUNRGBD/RGB_v8.pkl', 'rb'))
depth_gt = pickle.load(open('../pkls/SUNRGBD/Depth_v8.pkl', 'rb'))
rgb_keys = pickle.load(open('/data/jun/pkls/RGB/v10.pkl', 'rb'))
num_train = int(round(0.8 * len(rgb_keys)))
rgb_train_keys = rgb_keys[:num_train]
num_val = int(round((len(rgb_keys) - num_train)/2))
rgb_val_keys = rgb_keys[num_train:]
rgb_val_keys = rgb_val_keys[:num_val]
rgb_test_keys = rgb_keys[num_train:]
rgb_test_keys = rgb_test_keys[num_val:]


In [5]:
path_prefix = '/data/jun/dataset/'
rgb_inputs = []
depth_inputs = []
images = []
# img_path = path_prefix + sorted(val_keys )[100]
# img = image.load_img(img_path, target_size=(300, 300))
# img = image.img_to_array(img)
# images.append(imread(img_path))
# inputs.append(img.copy())
# inputs = preprocess_input(np.array(inputs))

for rgb_key in rgb_test_keys:
    rgb_img_path = path_prefix + rgb_key
    rgb_img = image.load_img(rgb_img_path, target_size=(300, 300))
    img = imread(rgb_img_path)
    images.append(img)
    rgb_img = image.img_to_array(rgb_img)
    rgb_inputs.append(rgb_img.copy())
    
inputs = preprocess_input(np.array(rgb_inputs))

/home/jun/.virtualenvs/rgbd-ssd/lib/python3.4/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app


In [6]:
priors = pickle.load(open('../pkls/prior_boxes_ssd300.pkl', 'rb'))
bbox_util = BBoxUtility(NUM_CLASSES, priors)
preds = model.predict(inputs, batch_size=16, verbose=1)
results = bbox_util.detection_out(preds,  confidence_threshold=0.50)

966/966 [==============================] - 14s    


In [7]:
gt_bboxes = []
gt_labels = []
gt_scores = []
for key in rgb_test_keys:
    index = np.where(rgb_gt[key][:, 4:] == 1)
    gt_bboxes.append(rgb_gt[key][:, :4])
    gt_labels.append((index[1]).reshape(len(index[1]), 1))
    gt_scores.append(np.ones((len(index[1]), 1)))
gt_bboxes = np.array(gt_bboxes)
gt_labels = np.array(gt_labels)
gt_scores = np.array(gt_scores)

In [8]:
pred_labels = []
pred_scores = []
pred_bboxes = []
for result in results:
    if len(result) != 0:
        nm = len(result[:, 1])
        #pred_labels.append((result[:, 0]-1).reshape(nm, 1))
        pred_labels.append((result[:, 0]-1).reshape(nm, 1))
        pred_scores.append(result[:, 1:2].reshape(nm, 1))
        pred_bboxes.append(result[:, 2:].reshape(nm, 4))
    else:
        pred_labels.append(np.array([]).reshape(0, 1))
        pred_scores.append(np.array([]).reshape(0, 1))
        pred_bboxes.append(np.array([]).reshape(0, 1))
pred_labels = np.array(pred_labels)
pred_scores = np.array(pred_scores)

pred_bboxes = np.array(pred_bboxes)

In [9]:
gt_labels.shape

(966,)

In [10]:
prec, rec, mean_iou = calc_detection_prec_rec(pred_labels, pred_scores, pred_bboxes, gt_bboxes, gt_labels, iou_thresh=0.5)

In [11]:
ap = calc_detection_ap(prec, rec, use_07_metric=True)
{'ap': ap, 'map': np.nanmean(ap)}

{'ap': array([ 0.18181818,  0.59324585,  0.43314383,  0.04545455,  0.44047985,
         0.32744182,  0.25499823,  0.07584372,  0.26603448,  0.21085237,
         0.17550497,  0.199683  ,  0.27016922,  0.32682633,  0.33088786,
         0.4048048 ,  0.41381903,  0.20234604,  0.77104176]),
 'map': 0.31181030921438413}

In [12]:
import csv

f_recall = open('rgb_recall.csv', 'w')
f_precision = open('rgb_precision.csv', 'w')
writer_recall = csv.writer(f_recall, lineterminator='\n')
writer_precision = csv.writer(f_precision, lineterminator='\n')

## PRC

In [ ]:
for i in range(len(NYU_CLASSES)):
    plt.figure()
    y = prec[i]
    x = rec[i]
    writer_recall.writerow(x)
    writer_precision.writerow(y)

f_recall.close()
f_precision.close()
#     plt.plot(x, y)
#     plt.axis([0, 1.0, 0, 1.0])
#     plt.title(NYU_CLASSES[i])
#     plt.xlabel('Recall')
#     plt.ylabel('Precision')
#     plt.savefig(('../results/PRC/RGB/' + NYU_CLASSES[i]+'.png'))

In [ ]:
mAP_array = []
for i in np.linspace(0, 1, 101):
    prec, rec, mean_iou = calc_detection_prec_rec(pred_labels, pred_scores, pred_bboxes, gt_bboxes, gt_labels, iou_thresh=i)
    ap = calc_detection_ap(prec, rec, use_07_metric=True)
    mAP_array.append(np.nanmean(ap))

In [ ]:
print(mAP_array)
plt.plot(np.linspace(0, 1, 101), np.array(mAP_array))
plt.title('Overlap Threshold and mAP')
plt.xlabel('Overlap Threshold')
plt.ylabel('mAP')
plt.savefig('../results/map_overlap/RGB.png')

In [ ]:
ap_array = np.zeros((len(NYU_CLASSES), len(np.linspace(0, 1, 101))))
for i, thresh in enumerate(np.linspace(0, 1, 101)):
    prec, rec, mean_iou = calc_detection_prec_rec(pred_labels, pred_scores, pred_bboxes, gt_bboxes, gt_labels, iou_thresh=thresh)
    ap = calc_detection_ap(prec, rec, use_07_metric=True)
    for k in range(len(NYU_CLASSES)):
        ap_array[k][i] = ap[k]

        

In [ ]:

for k in range(len(NYU_CLASSES)):
    plt.figure()
    plt.plot(np.linspace(0, 1, 101), np.array(ap_array[k]))
    plt.title(NYU_CLASSES[k])
    plt.xlabel('Overlap Threshold')
    plt.ylabel('Average Precision')
    plt.savefig(('../results/ap_overlap/RGB/'+NYU_CLASSES[k]+'.png'))

In [ ]:
images = results

In [ ]:
for i, img in enumerate(images):
    plt.figure()
    if len(results[i]) == 0:
        continue
    det_label = results[i][:, 0]
    det_conf = results[i][:, 1]
    det_xmin = results[i][:, 2]
    det_ymin = results[i][:, 3]
    det_xmax = results[i][:, 4]
    det_ymax = results[i][:, 5]

    # Get detections with confidence higher than 0.6.
    top_indices = [i for i, conf in enumerate(det_conf) if conf >= 0.6]

    top_conf = det_conf[top_indices]
    top_label_indices = det_label[top_indices].tolist()
    top_xmin = det_xmin[top_indices]
    top_ymin = det_ymin[top_indices]
    top_xmax = det_xmax[top_indices]
    top_ymax = det_ymax[top_indices]

    colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
    plt.imshow(img / 255.)
    currentAxis = plt.gca()


    for i in range(top_conf.shape[0]):
        xmin = int(round(top_xmin[i] * img.shape[1]))
        ymin = int(round(top_ymin[i] * img.shape[0]))
        xmax = int(round(top_xmax[i] * img.shape[1]))
        ymax = int(round(top_ymax[i] * img.shape[0]))
        score = top_conf[i]
        label = int(top_label_indices[i])
        label_name = NYU_CLASSES[label - 1]
        display_txt = '{:0.2f}, {}'.format(score, label_name)
        coords = (xmin, ymin), xmax-xmin, ymax-ymin
        color = colors[label]
        currentAxis.add_patch(plt.Rectangle(*coords, fill=False, edgecolor=color, linewidth=2))
        currentAxis.text(xmin, ymin, display_txt, bbox={'facecolor':color, 'alpha':0.5})
    
    plt.savefig('../results/detection_images/RGB/image' + str(i)+'_v10.png')

In [ ]:
y_true = []
for key in val_keys:
    y_true.append(gt[key])
y_true = np.array(y_true)
print(y_true.shape)

In [ ]:
inputs = []
images = []
for key in val_keys:
    img_path = path_prefix + key
    img = image.load_img(img_path, target_size=(300, 300))
    img = image.img_to_array(img)
    images.append(imread(img_path))
    inputs.append(img.copy())
inputs = preprocess_input(np.array(inputs))
preds = model.predict(inputs, batch_size=1, verbose=1)
results = bbox_util.detection_out(preds)

In [ ]:
#calc_map(y_true, results)
print(results[0])